In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
import json
import os
import time
import numpy as np
import pandas as pd
from datetime import datetime
from here_location_services import LS
from here_location_services.config.routing_config import ROUTING_RETURN
from here_los_utils import *

In [17]:
LS_API_KEY = os.environ.get("HERE_API_KEY")  # Get API KEY from environment.
ls = LS(api_key=LS_API_KEY)

### Load trips data

In [18]:
trips = pd.read_csv("../data/processed/trip_od_full.csv", low_memory=False)
trips.shape

(36287, 111)

In [19]:
# Use relavent columns
cols = ['TRIPID', 'Mode', 'LINKMODE', 'TRIPPURP', 'DESTPURP1', 'TRIPTIME', 'TRAVTIME',
        'TRAVDOW', 'TRAVDATE', 'TRAVMONTH', 'TRAVYEAR', 'STARTHOUR', 'STARTIME', 
        'HomeSA1', 'CUMDIST','ORIGSA1', 'DESTSA1', 'ORIG_X', 'ORIG_Y', 'DEST_X', 'DEST_Y']
trips = trips[cols]
trips.shape

(36287, 21)

### Get trips departure time

In [20]:
# Get detailed start time of trips
trips['STARTMIN'] = trips['STARTIME'] % 60
trips['START_DETAIL_TIME'] = (trips['TRAVDATE'].astype(str) + " " + trips['STARTHOUR'].astype(str) 
                              + ":" + trips['STARTMIN'].astype(str))

In [21]:
trips.head().T

,0,1,2,3,4
TRIPID,Y12H0000107P01T01,Y12H0000107P01T02,Y12H0000107P01T03,Y12H0000107P01T04,Y12H0000107P02T01
Mode,Car,Car,Walk,Walk,Car
LINKMODE,Vehicle Driver,Vehicle Driver,Walking,Walking,Vehicle Passenger
TRIPPURP,Social,Social,Recreational,Recreational,Social
DESTPURP1,Social,At or Go Home,Recreational,At or Go Home,Social
TRIPTIME,20,10,30,30,20
TRAVTIME,20,10,30,30,20
TRAVDOW,Sunday,Sunday,Sunday,Sunday,Sunday
TRAVDATE,27/05/2012,27/05/2012,27/05/2012,27/05/2012,27/05/2012
TRAVMONTH,May,May,May,May,May


### Collect Level of Service from HERE APIs

In [22]:
def get_datetime(str_time):
    return datetime.strptime(str_time,'%d/%m/%Y %H:%M')

# Test
print(get_datetime("27/05/2012 23:30"))

def get_trip_los(trip):
    time.sleep(0.3)
    start_time = get_datetime(trip['START_DETAIL_TIME'])
    orig = [trip.ORIG_Y, trip.ORIG_X]
    dest = [trip.DEST_Y, trip.DEST_X]
    (trip['CAR_TIME'], trip['CAR_LEN'], trip['CYCLE_TIME'], trip['CYCLE_LEN'], 
     trip['WALK_TIME'], trip['WALK_LEN']) = collect_here_los(ls, orig, dest, start_time)
    return trip

2012-05-27 23:30:00


In [23]:
trips.Mode.value_counts()

Walk     17115
Car      15764
Train     1704
Cycle     1704
Name: Mode, dtype: int64

In [24]:
# Divide data into walk_trips and not_walk_trips for smaller APIs number of queries
car_trips = trips[trips.Mode.isin(['Car'])]
walk_trips = trips[trips.Mode.isin(['Walk'])]
train_trips = trips[trips.Mode.isin(['Train'])]
cycle_trips = trips[trips.Mode.isin(['Cycle'])]
assert len(trips) == len(car_trips) + len(walk_trips) + len(train_trips) + len(cycle_trips)
walk_trips.shape, car_trips.shape, train_trips.shape, cycle_trips.shape, trips.shape

((17115, 23), (15764, 23), (1704, 23), (1704, 23), (36287, 23))

In [25]:
# Use batch processing as limitation of HERE service
# ApiError: 429, Too Many Requests, {"error":"Too Many Requests","error_description":"Rate limit for this service has been reached"}

def process_trips_los(trips_df, batch_size, save_path, df_name):
    """
    Process trips and get los from HERE APIs
    Parameters
        trips_df: trips dataframe
        batch_size: process in batch due to quota limitation of HERE APIs
        save_path: where to save collected batch files
    """
    batch_idxs = np.arange(len(trips_df)) // batch_size
    for gr, df in trips_df.groupby(batch_idxs):        
        trip_los = df.apply(lambda x: get_trip_los(x), axis=1)
        trip_los.to_csv(f"{save_path}/{df_name}_{gr}.csv")
        print(f"Finished batch {gr+1}", df.shape, trip_los.shape)
        time.sleep(5)

In [14]:
%%time
# RUN ONCE
save_path = "../data/los_here"
# batch_size = 100
# process_trips_los(train_trips, batch_size, save_path, "train_trips_los")
batch_size = 200
process_trips_los(cycle_trips, batch_size, save_path, "cycle_trips_los")

Finished batch 1 (200, 23) (200, 29)
Finished batch 2 (200, 23) (200, 29)
Finished batch 3 (200, 23) (200, 29)
Finished batch 4 (200, 23) (200, 29)
Finished batch 5 (200, 23) (200, 29)
Finished batch 6 (200, 23) (200, 29)
Finished batch 7 (200, 23) (200, 29)
Finished batch 8 (200, 23) (200, 29)
Finished batch 9 (104, 23) (104, 29)
CPU times: user 2min 36s, sys: 3.58 s, total: 2min 39s
Wall time: 47min 3s


In [26]:
%%time
# RUN ONCE
save_path = "../data/los_here"
batch_size = 200
process_trips_los(car_trips, batch_size, save_path, "car_trips_los")

Finished batch 1 (200, 23) (200, 29)
Finished batch 2 (200, 23) (200, 29)
Finished batch 3 (200, 23) (200, 29)
Finished batch 4 (200, 23) (200, 29)
Finished batch 5 (200, 23) (200, 29)
Finished batch 6 (200, 23) (200, 29)
Finished batch 7 (200, 23) (200, 29)
Finished batch 8 (200, 23) (200, 29)
Finished batch 9 (200, 23) (200, 29)
Finished batch 10 (200, 23) (200, 29)
Finished batch 11 (200, 23) (200, 29)
Finished batch 12 (200, 23) (200, 29)
Finished batch 13 (200, 23) (200, 29)
Finished batch 14 (200, 23) (200, 29)
Finished batch 15 (200, 23) (200, 29)
Finished batch 16 (200, 23) (200, 29)
Finished batch 17 (200, 23) (200, 29)
Finished batch 18 (200, 23) (200, 29)
Finished batch 19 (200, 23) (200, 29)
Finished batch 20 (200, 23) (200, 29)
Finished batch 21 (200, 23) (200, 29)
Finished batch 22 (200, 23) (200, 29)
Finished batch 23 (200, 23) (200, 29)
Finished batch 24 (200, 23) (200, 29)
Finished batch 25 (200, 23) (200, 29)
Finished batch 26 (200, 23) (200, 29)
Finished batch 27 (20

In [27]:
walk_trips.shape

(17115, 23)

In [ ]:
%%time
# RUN ONCE
save_path = "../data/los_here"
batch_size = 200
process_trips_los(walk_trips, batch_size, save_path, "walk_trips_los")

In [75]:
# THIS WILL RETURN ERROR
# # ApiError: 429, Too Many Requests, {"error":"Too Many Requests","error_description":"Rate limit for this service has been reached"}
# trip_los = trips.apply(lambda x: get_trip_los(x), axis=1)
# trip_los.shape